# Predictive maintenance of Lathe machine

#### Importing important libraries and modules

In [1]:
import pandas as pd
import numpy as np
import torch 
import tensorflow as tf
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

#### Importing Vibration Data and converting it to proper Time-series format.

In [4]:
dataframes = []  # To store the imported data from each file

for i in range(1, 61):
    file = f"Data/{i}.xlsx"
    df = pd.read_excel(file)  # Use pd.read_excel() for Excel files
    df = df.dropna(axis='columns', how='all')  # Drop empty columns
    df = df.dropna(axis='rows', how='all')  # Drop empty rows
    df.columns = ['Time', 'X', 'Y', 'Z']  # Assign column names explicitly
    df = df.iloc[1:]  # Exclude the original header row from the data
    df['Time'] = pd.to_datetime(df['Time'], unit='s').dt.time  # Convert 'Time' column to datetime
    dataframes.append(df)
    print(i) #too keep an eye on progress
exp = pd.read_excel("Data/Experiment Summary.xlsx")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [78]:
#view imported data.

for i in range(60):
    print(i, dataframes[i].shape, dataframes[i].head(5))

0 (98816, 4)               Time         X         Y         Z
1         00:00:00 -0.961714 -2.247089  -4.40107
2  00:00:00.000976 -0.394591 -2.298639 -1.552597
3  00:00:00.001953  -5.44346  0.849591   5.49498
4  00:00:00.002929  -1.46255   0.77963  1.958311
5  00:00:00.003906  1.148426   1.03738 -1.342826
1 (102400, 4)               Time         X         Y         Z
1         00:00:00 -1.212132  1.217805  3.375187
2  00:00:00.000976 -1.370485  1.744351  2.039275
3  00:00:00.001953  3.089165  2.274579 -0.801837
4  00:00:00.002929  0.120975  1.409276  -0.48534
5  00:00:00.003906 -0.928571 -0.203501  0.887373
2 (103424, 4)               Time         X         Y         Z
1         00:00:00 -3.554277  0.234673  1.156028
2  00:00:00.000976 -3.035028 -0.895744  2.050316
3  00:00:00.001953 -1.016954  1.600747 -0.095239
4  00:00:00.002929  -2.09596 -1.227136 -1.821252
5  00:00:00.003906  0.323519  1.103658 -1.497394
3 (97792, 4)               Time         X         Y         Z
1         00:00

In [5]:
# Time_domain plot


def time_domain(num):
    df = dataframes[num]

    # Create a line plot using Plotly
    fig = px.line(df, x='Time', y=['X', 'Y', 'Z'], title='Vibration Sensor Data', labels={'value':'acceleration'})
    
    # Display the plot
    fig.update_layout(
        height=600,
        showlegend=True,
        paper_bgcolor= 'darkgrey')
    fig.show()
    

# Calculating PSD

from scipy import signal

def calculate_psd(dataframe, fs=1000):
    time = dataframe['Time']
    x = dataframe['X']
    y = dataframe['Y']
    z = dataframe['Z']

    # Convert time values to seconds
    time_seconds = [(t.hour * 3600 + t.minute * 60 + t.second + t.microsecond / 1e6) for t in time]

    # Apply Hanning window function
    window = np.hanning(len(time_seconds))
    x_windowed = x * window
    y_windowed = y * window
    z_windowed = z * window

    # Calculate PSD using periodogram
    f, psd_x = signal.periodogram(x_windowed, fs)
    _, psd_y = signal.periodogram(y_windowed, fs)
    _, psd_z = signal.periodogram(z_windowed, fs)

    return f, psd_x, psd_y, psd_z



In [6]:



def freq_domain(frequencies, psd_x, psd_y, psd_z):
    # Convert complex PSD values to magnitude
    psd_x_mag = np.abs(psd_x)
    psd_y_mag = np.abs(psd_y)
    psd_z_mag = np.abs(psd_z)

    # Create line plots for X, Y, and Z axes
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=frequencies, y=psd_x_mag, mode='lines', name='X'))
    fig.add_trace(go.Scatter(x=frequencies, y=psd_y_mag, mode='lines', name='Y'))
    fig.add_trace(go.Scatter(x=frequencies, y=psd_z_mag, mode='lines', name='Z'))

    fig.update_layout(
        title='Power Spectral Density',
        xaxis=dict(title='Frequency'),
        yaxis=dict(title='acceleration'),
        showlegend=True,
        paper_bgcolor= 'darkgrey',
        height=600


    )

    fig.show()




### Time Domain and Frequency Domain charts:

##### RPM: 190

In [ ]:
# Calling the function with the provided DataFrame
for j in range(5):
    for i in range(j, 60, 20):
        print(exp.iloc[i])
        new = time_domain(i) 
        experiment, x, y, z = calculate_psd(dataframes[i], fs=1000)

        x = freq_domain(experiment, x, y, z)